In [5]:
import pandas as pd
import json
import http.client, urllib.parse
# import nbconvert
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import pipeline

In [6]:
import voila
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-11-06


In [8]:
# !jupyter serverextension enable voila --sys-prefix

In [9]:
model_name = "finbert_finetuned_train_on_kg.bin"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [10]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [11]:
# Image Widget

file = open("download.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='300'
                )

label_headline = widgets.Label(
                    value='Image by Iconka',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [12]:
# news source

source = widgets.Dropdown(
    options=[('General',''),('BBC', 'bbc'), ('CNN', 'cnn'), ('ESPN', 'espn')],
    value='',
    description='Source:',
)

In [13]:
# category

category = widgets.Dropdown(
    options=[('General', 'general'), ('Business', 'business'),('Entertainment','entertainment'),('Health', 'health'),('Science','science'),('Technology','technology')],
    value='general',
    description='Category:',
)

In [14]:
fill = widgets.Text(
    value='',
    placeholder='Enter Keyword(s)',
    description='Keyword:',
    disabled=False
)

In [15]:
#Date picker

date_start = widgets.DatePicker(value = today, description='Pick a Start')
date_end = widgets.DatePicker(value = today,description='Pick an End')

In [73]:
button_send = widgets.Button(
                description='Confirm',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        
        if(fill.value == ''):
            print("Analysing" ,category.value, "news from", date_start.value,"to",date_end.value)
        elif(fill.value != ''):
            print("Analysing" ,category.value, "news", "with",fill.value, "as a keyword from", date_start.value,"to",date_end.value)
        temp_start = date_start.value.strftime('%Y-%m-%d')
        temp_end = date_end.value.strftime('%Y-%m-%d')
        parsed_date = temp_start +','+temp_end
        ACCESS_KEY = '243247ebaf607124b6b062d06e319452'

        conn = http.client.HTTPConnection('api.mediastack.com')

        params = urllib.parse.urlencode({
    'access_key': ACCESS_KEY,
    'languages' : 'en',
    'limit' : '100',
    'date' : parsed_date,
    'categories' : category.value,
    'keywords' : fill.value,
    'sources' : source.value,
    'sort' : 'published_desc'
    })

        conn.request('GET', '/v1/news?{}'.format(params))

        res = conn.getresponse()
        data = res.read()
        dict_data = json.loads(data.decode('utf-8'))
        if(dict_data['pagination']['count'] != 0):
            df = pd.DataFrame()
            for x in dict_data['data']:
                df = df.append(x, ignore_index=True)
            title_df = df['title']
            title_df
            display(title_df)
            print("Loading...")
            tabulation_df = pd.DataFrame()
            neg_count = 0
            pos_count = 0
            for i in title_df:
                if(classifier(i)[0]['label'] == 'negative'):
                    neg_count+= 1
                    
                elif(classifier(i)[0]['label'] == 'positive'):
                    pos_count+= 1
            print('Analysis complete!')        
            print('The ratio of negative to positive is:' ,neg_count,'-',pos_count)
            if(neg_count>pos_count):
                print('Therefore, the general sentiment is negative.')
            else:
                print('Therefore, the general sentiment is positive.')
                
            
        elif (dict_data['pagination']['count'] == 0):
            print("Not enough data for these parameters. Please try again!")
        
        
        
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [74]:
vbox_text = widgets.VBox([source, category, fill, date_start, date_end, vbox_result])

In [75]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)